In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl
/kaggle/input/torch_geometric-2.5.3-py3-none-any.whl
/kaggle/input/torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl
/kaggle/input/torch_spline_conv-1.2.2-cp310-cp310-linux_x86_64.whl
/kaggle/input/torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl


In [2]:
!git clone https://github.com/rampasek/GraphGPS.git

Cloning into 'GraphGPS'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 526 (delta 254), reused 225 (delta 225), pack-reused 210 (from 1)
Receiving objects: 100% (526/526), 12.93 MiB | 45.65 MiB/s, done.
Resolving deltas: 100% (336/336), done.


In [3]:
%cd /kaggle/working/GraphGPS
!ls 

/kaggle/working/GraphGPS
GraphGPS.png  README.md  final-results.zip  main.py  setup.py  unittests
LICENSE       configs	 graphgps	    run      tests


##  installation for slurm if needed using conda 

In [4]:
# # for slurm if needed using conda 
# conda create -n graphgps python=3.10
# conda activate graphgps

# conda install pytorch=1.13 torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia
# conda install pyg=2.2 -c pyg -c conda-forge
# pip install pyg-lib -f https://data.pyg.org/whl/torch-1.13.0+cu117.html

# # RDKit is required for OGB-LSC PCQM4Mv2 and datasets derived from it.  
# conda install openbabel fsspec rdkit -c conda-forge

# pip install pytorch-lightning yacs torchmetrics
# pip install performer-pytorch
# pip install tensorboardX
# pip install ogb
# pip install wandb

# conda clean --all

# installation for kaggle

In [9]:
# Check existing environment
import sys
print("Python version:", sys.version)
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118

!pip install torch_geometric

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu124.html
# Install other required packages
!pip install pytorch-lightning yacs torchmetrics
!pip install performer-pytorch
!pip install tensorboardX
!pip install ogb
!pip install wandb

# Attempt to install RDKit (may have limitations)
!pip install rdkit-pypi
# !pip install openbabel fsspec
!pip install  fsspec

# Clean up pip cache (optional)
!pip cache purge


Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
PyTorch version: 2.4.0
CUDA available: True
CUDA version: 12.3
Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torchaudio 2.4.0
Uninstalling torchaudio-2.4.0:
  Successfully uninstalled torchaudio-2.4.0
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 392.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 61.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 84.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 22.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.3 MB/s eta 

In [10]:
import sys
print("Python version:", sys.version)
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
PyTorch version: 2.4.0
CUDA available: True
CUDA version: 12.3


In [11]:
import torch_geometric
print("PyG version:", torch_geometric.__version__)

import pytorch_lightning
print("PyTorch Lightning version:", pytorch_lightning.__version__)

import rdkit
print("RDKit version:", rdkit.__version__)


/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/libpyg.so: undefined symbol: _ZNK5torch8autograd4Node4nameEv
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings

PyG version: 2.6.0
PyTorch Lightning version: 2.4.0
RDKit version: 2022.09.5


In [12]:
print("Torch version:", torch.__version__)

Torch version: 2.4.0


In [13]:
import random
import numpy as np
import torch

def seed_everything(seed=42):
    """
    Seed all sources of randomness to make all runs reproducible.
    """
    random.seed(seed)  # Python's random library
    np.random.seed(seed)  # NumPy library
    torch.manual_seed(seed)  # PyTorch
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
#         torch.backends.cudnn.deterministic = True
#         torch.backends.cudnn.benchmark = False

# Usage
seed_everything(42)  # You can change '42' to any seed number you prefer


In [9]:
%cd /kaggle/working/GraphGPS
!WANDB_MODE=disabled python main.py --cfg configs/GPS/ogbg-molhiv-GPS.yaml

/kaggle/working/GraphGPS
[*] Run ID 0: seed=0, split_index=0
    Starting now: 2024-09-23 13:16:20.178181
Downloaded 0.00 GB: 100%|█████████████████████████| 3/3 [00:00<00:00,  3.88it/s]
Extracting ./datasets/hiv.zip
Processing...
Loading necessary files...
This might take a while.
Processing graphs...
100%|██████████████████████████████████| 41127/41127 [00:00<00:00, 60420.42it/s]
Converting graphs into PyG objects...
100%|██████████████████████████████████| 41127/41127 [00:02<00:00, 18891.90it/s]
Saving...
Done!
/opt/conda/lib/python3.10/site-packages/ogb/graphproppred/dataset_pyg.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

# Our Model - WeightedGraphGPS

In [28]:
#clone for our project
%cd /kaggle/working
!git clone --branch add-relur https://github.com/omertalmi5/WeightedGraphGPS.git

/kaggle/working
Cloning into 'WeightedGraphGPS'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 526 (delta 257), reused 222 (delta 222), pack-reused 208 (from 1)
Receiving objects: 100% (526/526), 12.93 MiB | 41.36 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [7]:
!pip install wandb

In [7]:
!wandb login 5f1c292fde40243d281618fe88d712346cfe90d7

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [23]:
import os
# Set the environment variable
os.environ['USERNAME'] = "omertalmi-tel-aviv-university"

In [29]:
%cd /kaggle/working/WeightedGraphGPS
!WANDB_MODE=online python main.py --cfg configs/GPS/ogbg-molhiv-GPS.yaml wandb.use True wandb.entity $USERNAME

/kaggle/working/WeightedGraphGPS


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


[*] Run ID 0: seed=0, split_index=0
    Starting now: 2024-09-24 07:24:18.450063
Downloaded 0.00 GB: 100%|█████████████████████████| 3/3 [00:00<00:00,  9.87it/s]
Extracting ./datasets/hiv.zip
Processing...
Loading necessary files...
This might take a while.
Processing graphs...
100%|██████████████████████████████████| 41127/41127 [00:00<00:00, 63760.53it/s]
Converting graphs into PyG objects...
100%|██████████████████████████████████| 41127/41127 [00:01<00:00, 20980.75it/s]
Saving...
Done!
/opt/conda/lib/python3.10/site-packages/ogb/graphproppred/dataset_pyg.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


 76%|███████████████████████████▌        | 31451/41127 [00:40<00:12, 784.35it/s]

...

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


train: {'epoch': 99, 'time_epoch': 80.81963, 'eta': 0.0, 'eta_hours': 0.0, 'loss': 0.07397405, 'lr': 3e-08, 'params': 559955, 'time_iter': 0.07854, 'accuracy': 0.97629, 'precision': 0.80623, 'recall': 0.48295, 'f1': 0.60406, 'auc': 0.95746}
val: {'epoch': 99, 'time_epoch': 3.97894, 'loss': 0.09423107, 'lr': 0, 'params': 559955, 'time_iter': 0.03084, 'accuracy': 0.97666, 'precision': 0.39726, 'recall': 0.35802, 'f1': 0.37662, 'auc': 0.78409}
test: {'epoch': 99, 'time_epoch': 4.03226, 'loss': 0.1750503, 'lr': 0, 'params': 559955, 'time_iter': 0.03126, 'accuracy': 0.96377, 'precision': 0.37333, 'recall': 0.21538, 'f1': 0.27317, 'auc': 0.69773}
> Epoch 99: took 89.0s (avg 90.6s) | Best so far: epoch 14	train_loss: 0.1222 train_auc: 0.8179	val_loss: 0.0789 val_auc: 0.8184	test_loss: 0.1182 test_auc: 0.7462
Avg time per epoch: 90.56s
Total train loop time: 2.52h


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:       best/epoch ▁▂▃▅▆▇██████████████████████████████████
wandb:    best/test_auc ▁█▇▇████████████████████████████████████
wandb:   best/test_loss █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:   best/train_auc ▁▄▆▆▇███████████████████████████████████
wandb:  best/train_loss █▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:     best/val_auc ▁▆▇▇████████████████████████████████████
wandb:    best/val_loss █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:    test/accuracy ▁███████████████████████████████████████
wandb:         test/auc ▁█▇▇███████▇▇▇▇▆▇▇▇▆▇▇▇▆▇▇▇▇▇▇▆▆▆▆▇▆▆▇▇▆
wandb:       test/epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:          test/f1 ▂▁▇▆▇██▆▇▇▇▇▇▆▆▆▆▆▇▇▇▇▇▆▆▆▅▅▇▆▆▆▅▆▆▇▆▆▆▆
wandb:        test/loss █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂
wandb:          test/lr ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:      test/params ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/omertalmi-tel-aviv-university/molhiv/k5dhq0ta/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/omertalmi-tel-aviv-university/molhiv/k5dhq0ta/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/omertalmi-tel-aviv-university/molhiv/k5dhq0ta/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/omertalmi-tel-aviv-university/molhiv/k5dhq0ta/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
dropped chunk 404 Client Error: